In [ ]:
!pip install pyspark
from datetime import datetime
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import pyspark.pandas as ps
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession,SQLContext
from pyspark import SparkContext
from pyspark import SparkConf
import pyspark.pandas as ps
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.types import IntegerType
from pyspark.ml.evaluation import RegressionEvaluator


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
     .builder \
     .appName("ubiquant") \
     .getOrCreate()   

In [ ]:
df=spark.read.csv("/kaggle/input/ubiquant-market-prediction/train.csv", inferSchema=True, header=True)

In [ ]:
df=df.drop('time_id','investment_id','row_id')

In [ ]:
from pyspark.ml.feature import VectorAssembler
feature_list = []
for col in df.columns:
    if col == 'target':
        continue
    else:
        feature_list.append(col)
assembler = VectorAssembler(inputCols=feature_list, outputCol="features")
df = assembler.transform(df)

In [ ]:
train, test = df.randomSplit([0.9, 0.1], seed = 42)


In [ ]:

rf = RandomForestRegressor(featuresCol = 'features', labelCol = 'target')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)

In [ ]:
pred=predictions.select('features','prediction').show(10)

In [ ]:

from pyspark.ml.evaluation import RegressionEvaluator


In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

In [ ]:
unlist = udf(lambda x: float(list(x)[0]), DoubleType())

predictions = predictions.withColumn("dfeatures", unlist("features"))

In [ ]:
evaluator = RegressionEvaluator(labelCol="dfeatures", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

In [ ]:
print(rmse)

In [ ]:
from pyspark.ml.stat import Correlation

In [ ]:
#pearsonCorr = Correlation.corr(predictions, 'dfeatures', 'pearson').collect()[0][0]

In [ ]:
#spark.stop